In [1]:
import pandas as pd
from nba_api.stats.static import teams
import json

from nba_api.stats.endpoints._base import Endpoint
from nba_api.stats.library.http import NBAStatsHTTP
from nba_api.stats.library.parameters import GameScopeDetailed, LeagueID, PlayerOrTeam, PlayerScope, Season, SeasonType

from bs4 import BeautifulSoup as bs
from splinter import Browser
import datetime as dt
import requests
from urllib.request import urlopen

In [2]:
nba_teams = teams.get_teams()
print('Number of teams fetched: {}'.format(len(nba_teams)))
nba_teams[:5]

Number of teams fetched: 30


[{'id': 1610612737,
  'full_name': 'Atlanta Hawks',
  'abbreviation': 'ATL',
  'nickname': 'Hawks',
  'city': 'Atlanta',
  'state': 'Atlanta',
  'year_founded': 1949},
 {'id': 1610612738,
  'full_name': 'Boston Celtics',
  'abbreviation': 'BOS',
  'nickname': 'Celtics',
  'city': 'Boston',
  'state': 'Massachusetts',
  'year_founded': 1946},
 {'id': 1610612739,
  'full_name': 'Cleveland Cavaliers',
  'abbreviation': 'CLE',
  'nickname': 'Cavaliers',
  'city': 'Cleveland',
  'state': 'Ohio',
  'year_founded': 1970},
 {'id': 1610612740,
  'full_name': 'New Orleans Pelicans',
  'abbreviation': 'NOP',
  'nickname': 'Pelicans',
  'city': 'New Orleans',
  'state': 'Louisiana',
  'year_founded': 2002},
 {'id': 1610612741,
  'full_name': 'Chicago Bulls',
  'abbreviation': 'CHI',
  'nickname': 'Bulls',
  'city': 'Chicago',
  'state': 'Illinois',
  'year_founded': 1966}]

In [3]:
#year = 2020
url = "https://www.basketball-reference.com/leagues/NBA_2020_ratings.html"
html = urlopen(url)
soup = bs(html)

In [4]:
#Find Headers
soup.findAll('tr', limit=2)
headers = [th.getText() for th in soup.findAll('tr', limit=2)[1].findAll('th')]
headers = headers[1:]
headers

['Team',
 'Conf',
 'Div',
 'W',
 'L',
 'W/L%',
 'MOV',
 'ORtg',
 'DRtg',
 'NRtg',
 'MOV/A',
 'ORtg/A',
 'DRtg/A',
 'NRtg/A']

In [5]:
# Scrape Data
rows = soup.findAll('tr')[1:]
team_stats = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]

In [6]:
stats_2020 = pd.DataFrame(team_stats, columns = headers)
stats_2020.head(33)

,Team,Conf,Div,W,L,W/L%,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,Milwaukee Bucks,E,C,35,6,.854,12.29,114.26,102.47,11.79,11.18,114.02,103.33,10.69
2,Los Angeles Lakers,W,P,32,7,.821,8.28,113.56,105.48,8.09,8.57,113.17,104.78,8.40
3,Dallas Mavericks,W,SW,24,15,.615,6.87,117.58,110.64,6.94,7.18,117.89,110.68,7.22
4,Los Angeles Clippers,W,P,27,12,.692,6.10,112.68,106.77,5.91,6.47,112.35,106.07,6.28
5,Boston Celtics,E,A,26,11,.703,6.97,112.78,105.72,7.06,5.82,112.00,106.09,5.90
6,Toronto Raptors,E,A,25,13,.658,5.26,110.50,105.21,5.29,5.63,110.84,105.20,5.64
7,Houston Rockets,W,SW,26,12,.684,5.50,115.27,109.84,5.43,5.03,115.12,110.14,4.97
8,Philadelphia 76ers,E,A,25,15,.625,3.48,110.41,106.94,3.46,3.65,110.25,106.60,3.65
9,Utah Jazz,W,NW,26,12,.684,3.50,110.81,107.08,3.73,3.17,111.29,107.91,3.37


In [40]:
stats_2020.to_csv('stats_2020.csv')

In [7]:
url = "https://www.basketball-reference.com/leagues/NBA_2019_ratings.html"
html = urlopen(url)
soup = bs(html)

soup.findAll('tr', limit=2)
headers = [th.getText() for th in soup.findAll('tr', limit=2)[1].findAll('th')]
headers = headers[1:]

rows = soup.findAll('tr')[1:]
team_stats = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]

stats_2019 = pd.DataFrame(team_stats, columns = headers)
stats_2019.head(33)

,Team,Conf,Div,W,L,W/L%,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,Milwaukee Bucks,E,C,60,22,.732,8.87,114.23,105.76,8.47,8.05,113.89,106.23,7.66
2,Golden State Warriors,W,P,57,25,.695,6.46,116.63,110.24,6.39,6.42,116.60,110.24,6.37
3,Toronto Raptors,E,A,58,24,.707,6.09,113.99,108.00,5.99,5.49,113.78,108.40,5.38
4,Utah Jazz,W,NW,50,32,.610,5.26,111.35,106.11,5.24,5.29,111.23,105.94,5.28
5,Houston Rockets,W,SW,53,29,.646,4.77,116.25,111.43,4.82,4.97,116.45,111.43,5.03
6,Portland Trail Blazers,W,NW,53,29,.646,4.20,115.45,111.07,4.38,4.44,115.63,111.01,4.62
7,Denver Nuggets,W,NW,54,28,.659,3.95,113.76,109.62,4.14,4.20,113.74,109.35,4.39
8,Boston Celtics,E,A,49,33,.598,4.44,112.78,108.35,4.43,3.90,112.57,108.69,3.88
9,Oklahoma City Thunder,W,NW,49,33,.598,3.40,110.85,107.53,3.31,3.56,110.71,107.22,3.49


In [41]:
stats_2019.to_csv('stats_2019.csv')

In [8]:
url = "https://www.basketball-reference.com/leagues/NBA_2018_ratings.html"
html = urlopen(url)
soup = bs(html)

soup.findAll('tr', limit=2)
headers = [th.getText() for th in soup.findAll('tr', limit=2)[1].findAll('th')]
headers = headers[1:]

rows = soup.findAll('tr')[1:]
team_stats = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]

stats_2018 = pd.DataFrame(team_stats, columns = headers)
stats_2018.head(6)

,Team,Conf,Div,W,L,W/L%,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,Houston Rockets,W,SW,65,17,.793,8.48,115.48,106.84,8.64,8.21,115.45,107.08,8.37
2,Toronto Raptors,E,A,59,23,.720,7.78,114.60,106.56,8.03,7.30,114.50,106.96,7.54
3,Golden State Warriors,W,P,58,24,.707,5.98,114.31,108.43,5.88,5.79,114.33,108.62,5.70
4,Utah Jazz,W,NW,48,34,.585,4.30,109.07,104.56,4.51,4.47,108.92,104.24,4.68
5,Philadelphia 76ers,E,A,52,30,.634,4.50,110.28,105.77,4.51,4.30,110.12,105.81,4.31


In [42]:
stats_2018.to_csv('stats_2018.csv')

In [9]:
url = "https://www.basketball-reference.com/leagues/NBA_2017_ratings.html"
html = urlopen(url)
soup = bs(html)

soup.findAll('tr', limit=2)
headers = [th.getText() for th in soup.findAll('tr', limit=2)[1].findAll('th')]
headers = headers[1:]

rows = soup.findAll('tr')[1:]
team_stats = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]

stats_2017 = pd.DataFrame(team_stats, columns = headers)
stats_2017.head(6)

,Team,Conf,Div,W,L,W/L%,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,Golden State Warriors,W,P,67,15,.817,11.63,116.26,104.57,11.69,11.35,116.02,104.61,11.41
2,San Antonio Spurs,W,SW,61,21,.744,7.20,111.81,104.25,7.56,7.13,111.62,104.12,7.49
3,Houston Rockets,W,SW,55,27,.671,5.77,115.59,109.96,5.63,5.84,115.70,109.98,5.72
4,Los Angeles Clippers,W,P,51,31,.622,4.29,113.38,108.88,4.50,4.42,113.46,108.82,4.64
5,Utah Jazz,W,NW,51,31,.622,3.94,110.46,106.19,4.27,4.00,110.31,105.98,4.33


In [43]:
stats_2017.to_csv('stats_2017.csv')

In [10]:
url = "https://www.basketball-reference.com/leagues/NBA_2016_ratings.html"
html = urlopen(url)
soup = bs(html)

soup.findAll('tr', limit=2)
headers = [th.getText() for th in soup.findAll('tr', limit=2)[1].findAll('th')]
headers = headers[1:]

rows = soup.findAll('tr')[1:]
team_stats = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]

stats_2016 = pd.DataFrame(team_stats, columns = headers)
stats_2016.head(6)

,Team,Conf,Div,W,L,W/L%,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,San Antonio Spurs,W,SW,67,15,.817,10.63,111.04,99.55,11.49,10.28,110.59,99.49,11.10
2,Golden State Warriors,W,P,73,9,.890,10.76,115.18,104.32,10.86,10.38,115.01,104.53,10.49
3,Oklahoma City Thunder,W,NW,55,27,.671,7.28,113.76,106.22,7.54,7.09,113.64,106.29,7.35
4,Cleveland Cavaliers,E,C,57,25,.695,5.99,111.80,105.32,6.48,5.44,111.75,105.85,5.90
5,Toronto Raptors,E,A,56,26,.683,4.50,111.11,106.16,4.95,4.08,111.18,106.68,4.50


In [44]:
stats_2016.to_csv('stats_2016.csv')

In [11]:
url = "https://www.basketball-reference.com/leagues/NBA_2015_ratings.html"
html = urlopen(url)
soup = bs(html)

soup.findAll('tr', limit=2)
headers = [th.getText() for th in soup.findAll('tr', limit=2)[1].findAll('th')]
headers = headers[1:]

rows = soup.findAll('tr')[1:]
team_stats = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]

stats_2015 = pd.DataFrame(team_stats, columns = headers)
stats_2015.head(6)

,Team,Conf,Div,W,L,W/L%,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,Golden State Warriors,W,P,67,15,.817,10.10,112.18,101.86,10.32,10.01,111.90,101.67,10.23
2,Los Angeles Clippers,W,P,56,26,.683,6.59,113.21,106.22,6.98,6.80,113.35,106.14,7.21
3,San Antonio Spurs,W,SW,55,27,.671,6.20,109.25,102.61,6.64,6.34,109.09,102.30,6.79
4,Atlanta Hawks,E,SE,60,22,.732,5.43,110.00,103.98,6.02,4.75,109.85,104.56,5.29
5,Portland Trail Blazers,W,NW,51,31,.622,4.23,108.81,104.30,4.50,4.41,108.72,104.03,4.69


In [45]:
stats_2015.to_csv('stats_2015.csv')

In [12]:
url = "https://www.basketball-reference.com/leagues/NBA_2014_ratings.html"
html = urlopen(url)
soup = bs(html)

soup.findAll('tr', limit=2)
headers = [th.getText() for th in soup.findAll('tr', limit=2)[1].findAll('th')]
headers = headers[1:]

rows = soup.findAll('tr')[1:]
team_stats = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]

stats_2014 = pd.DataFrame(team_stats, columns = headers)
stats_2014.head(6)

,Team,Conf,Div,W,L,W/L%,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,San Antonio Spurs,W,SW,62,20,.756,7.72,111.09,102.93,8.16,7.99,110.98,102.53,8.45
2,Los Angeles Clippers,W,P,57,25,.695,6.98,112.89,105.64,7.25,7.27,112.88,105.33,7.56
3,Oklahoma City Thunder,W,NW,59,23,.720,6.34,111.32,104.60,6.71,6.65,111.29,104.26,7.04
4,Golden State Warriors,W,P,51,31,.622,4.80,108.05,103.07,4.98,5.15,107.93,102.59,5.34
5,Houston Rockets,W,SW,54,28,.659,4.56,111.72,107.07,4.65,5.06,111.82,106.65,5.17


In [46]:
stats_2014.to_csv('stats_2014.csv')

In [13]:
url = "https://www.basketball-reference.com/leagues/NBA_2013_ratings.html"
html = urlopen(url)
soup = bs(html)

soup.findAll('tr', limit=2)
headers = [th.getText() for th in soup.findAll('tr', limit=2)[1].findAll('th')]
headers = headers[1:]

rows = soup.findAll('tr')[1:]
team_stats = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]

stats_2013 = pd.DataFrame(team_stats, columns = headers)
stats_2013.head(6)

,Team,Conf,Div,W,L,W/L%,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,Oklahoma City Thunder,W,NW,60,22,.732,9.21,113.30,103.35,9.95,9.15,113.21,103.33,9.89
2,Miami Heat,E,SE,66,16,.805,7.87,113.43,104.77,8.67,7.03,113.28,105.53,7.75
3,San Antonio Spurs,W,SW,58,24,.707,6.40,109.11,102.26,6.85,6.67,109.05,101.90,7.14
4,Los Angeles Clippers,W,P,56,26,.683,6.45,111.62,104.47,7.16,6.43,111.58,104.44,7.13
5,Denver Nuggets,W,NW,57,25,.695,5.09,111.23,105.97,5.26,5.37,111.34,105.76,5.58


In [47]:
stats_2013.to_csv('stats_2013.csv')

In [14]:
url = "https://www.basketball-reference.com/leagues/NBA_2012_ratings.html"
html = urlopen(url)
soup = bs(html)

soup.findAll('tr', limit=2)
headers = [th.getText() for th in soup.findAll('tr', limit=2)[1].findAll('th')]
headers = headers[1:]

rows = soup.findAll('tr')[1:]
team_stats = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]

stats_2012 = pd.DataFrame(team_stats, columns = headers)
stats_2012.head(6)

,Team,Conf,Div,W,L,W/L%,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,Chicago Bulls,E,C,50,16,.758,8.18,108.22,98.89,9.33,7.43,108.21,99.72,8.49
2,San Antonio Spurs,W,SW,50,16,.758,7.17,111.45,103.83,7.62,7.28,111.35,103.60,7.75
3,Oklahoma City Thunder,W,NW,47,19,.712,6.12,110.33,103.77,6.56,6.44,110.30,103.37,6.93
4,Miami Heat,E,SE,46,20,.697,5.98,107.66,101.10,6.56,5.72,107.97,101.71,6.26
5,Philadelphia 76ers,E,A,35,31,.530,4.24,104.70,100.00,4.70,3.59,104.42,100.45,3.97


In [48]:
stats_2012.to_csv('stats_2012.csv')

In [15]:
url = "https://www.basketball-reference.com/leagues/NBA_2011_ratings.html"
html = urlopen(url)
soup = bs(html)

soup.findAll('tr', limit=2)
headers = [th.getText() for th in soup.findAll('tr', limit=2)[1].findAll('th')]
headers = headers[1:]

rows = soup.findAll('tr')[1:]
team_stats = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]

stats_2011 = pd.DataFrame(team_stats, columns = headers)
stats_2011.head(6)

,Team,Conf,Div,W,L,W/L%,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,Miami Heat,E,SE,58,24,.707,7.46,112.62,104.35,8.27,6.76,112.43,104.94,7.49
2,Chicago Bulls,E,C,62,20,.756,7.32,109.30,101.07,8.23,6.53,109.02,101.65,7.37
3,Los Angeles Lakers,W,P,57,25,.695,6.11,111.61,105.00,6.61,6.01,111.40,104.89,6.51
4,San Antonio Spurs,W,SW,61,21,.744,5.71,112.31,106.23,6.07,5.86,112.22,105.97,6.25
5,Orlando Magic,E,SE,52,30,.634,5.46,108.78,102.87,5.91,4.92,108.63,103.31,5.32


In [49]:
stats_2011.to_csv('stats_2011.csv')

In [16]:
url = "https://www.basketball-reference.com/leagues/NBA_2010_ratings.html"
html = urlopen(url)
soup = bs(html)

soup.findAll('tr', limit=2)
headers = [th.getText() for th in soup.findAll('tr', limit=2)[1].findAll('th')]
headers = headers[1:]

rows = soup.findAll('tr')[1:]
team_stats = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]

stats_2010 = pd.DataFrame(team_stats, columns = headers)
stats_2010.head(6)

,Team,Conf,Div,W,L,W/L%,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,Orlando Magic,E,SE,59,23,.720,7.49,112.13,104.00,8.13,7.12,112.02,104.28,7.73
2,Cleveland Cavaliers,E,C,61,21,.744,6.52,112.27,105.07,7.21,6.17,112.29,105.47,6.82
3,Utah Jazz,W,NW,53,29,.646,5.34,111.57,105.81,5.76,5.33,111.44,105.69,5.75
4,San Antonio Spurs,W,SW,50,32,.610,5.09,110.61,105.17,5.44,5.07,110.49,105.07,5.42
5,Los Angeles Lakers,W,P,57,25,.695,4.72,109.51,104.57,4.93,4.78,109.32,104.32,5.00


In [50]:
stats_2010.to_csv('stats_2010.csv')

In [17]:
url = "https://www.basketball-reference.com/leagues/NBA_2009_ratings.html"
html = urlopen(url)
soup = bs(html)

soup.findAll('tr', limit=2)
headers = [th.getText() for th in soup.findAll('tr', limit=2)[1].findAll('th')]
headers = headers[1:]

rows = soup.findAll('tr')[1:]
team_stats = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]

stats_2009 = pd.DataFrame(team_stats, columns = headers)
stats_2009.head(6)

,Team,Conf,Div,W,L,W/L%,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,Cleveland Cavaliers,E,C,66,16,.805,8.93,113.29,103.20,10.09,8.68,113.18,103.37,9.80
2,Boston Celtics,E,A,62,20,.756,7.51,111.25,103.02,8.24,7.44,111.21,103.05,8.16
3,Los Angeles Lakers,W,P,65,17,.793,7.66,113.43,105.29,8.14,7.11,113.00,105.45,7.55
4,Orlando Magic,E,SE,59,23,.720,6.70,109.66,102.58,7.08,6.49,109.52,102.66,6.86
5,Portland Trail Blazers,W,NW,54,28,.659,5.34,114.74,108.68,6.06,5.00,114.62,108.94,5.67


In [51]:
stats_2009.to_csv('stats_2009.csv')

In [18]:
url = "https://www.basketball-reference.com/leagues/NBA_2008_ratings.html"
html = urlopen(url)
soup = bs(html)

soup.findAll('tr', limit=2)
headers = [th.getText() for th in soup.findAll('tr', limit=2)[1].findAll('th')]
headers = headers[1:]

rows = soup.findAll('tr')[1:]
team_stats = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]

stats_2008 = pd.DataFrame(team_stats, columns = headers)
stats_2008.head(6)

,Team,Conf,Div,W,L,W/L%,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,Boston Celtics,E,A,66,16,.805,10.26,111.10,99.75,11.35,9.31,110.55,100.23,10.32
2,Los Angeles Lakers,W,P,57,25,.695,7.26,113.71,106.08,7.63,7.34,113.72,105.99,7.73
3,Detroit Pistons,E,C,59,23,.720,7.39,112.09,103.64,8.45,6.67,111.76,104.09,7.66
4,Utah Jazz,W,NW,54,28,.659,6.88,114.52,107.05,7.47,6.87,114.60,107.15,7.44
5,New Orleans Hornets,W,SW,56,26,.683,5.28,112.21,106.44,5.77,5.46,112.30,106.34,5.96


In [52]:
stats_2008.to_csv('stats_2008.csv')

In [19]:
url = "https://www.basketball-reference.com/leagues/NBA_2007_ratings.html"
html = urlopen(url)
soup = bs(html)

soup.findAll('tr', limit=2)
headers = [th.getText() for th in soup.findAll('tr', limit=2)[1].findAll('th')]
headers = headers[1:]

rows = soup.findAll('tr')[1:]
team_stats = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]

stats_2007 = pd.DataFrame(team_stats, columns = headers)
stats_2007.head(6)

,Team,Conf,Div,W,L,W/L%,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,San Antonio Spurs,W,SW,58,24,.707,8.43,110.11,100.64,9.47,8.35,109.84,100.47,9.37
2,Dallas Mavericks,W,SW,67,15,.817,7.22,112.32,104.12,8.20,7.28,112.23,103.98,8.25
3,Phoenix Suns,W,P,61,21,.744,7.30,114.68,106.90,7.78,7.27,114.66,106.90,7.76
4,Houston Rockets,W,SW,52,30,.634,4.85,106.64,101.31,5.33,5.04,106.43,100.90,5.53
5,Chicago Bulls,E,C,49,33,.598,5.01,105.71,100.32,5.39,4.52,105.36,100.50,4.86


In [53]:
stats_2007.to_csv('stats_2007.csv')

In [20]:
url = "https://www.basketball-reference.com/leagues/NBA_2006_ratings.html"
html = urlopen(url)
soup = bs(html)

soup.findAll('tr', limit=2)
headers = [th.getText() for th in soup.findAll('tr', limit=2)[1].findAll('th')]
headers = headers[1:]

rows = soup.findAll('tr')[1:]
team_stats = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]

stats_2006 = pd.DataFrame(team_stats, columns = headers)
stats_2006.head(6)

,Team,Conf,Div,W,L,W/L%,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,San Antonio Spurs,W,SW,63,19,.768,6.82,108.02,100.32,7.70,6.69,107.93,100.38,7.54
2,Detroit Pistons,E,C,64,18,.780,6.67,111.59,103.90,7.69,6.23,111.35,104.15,7.20
3,Dallas Mavericks,W,SW,60,22,.732,6.07,112.87,105.73,7.14,5.96,113.06,106.07,6.99
4,Phoenix Suns,W,P,54,28,.659,5.55,112.27,106.45,5.82,5.48,112.32,106.58,5.74
5,Memphis Grizzlies,W,SW,49,33,.598,3.70,107.11,102.69,4.42,3.74,107.10,102.66,4.44


In [54]:
stats_2006.to_csv('stats_2006.csv')

In [21]:
url = "https://www.basketball-reference.com/leagues/NBA_2005_ratings.html"
html = urlopen(url)
soup = bs(html)

soup.findAll('tr', limit=2)
headers = [th.getText() for th in soup.findAll('tr', limit=2)[1].findAll('th')]
headers = headers[1:]

rows = soup.findAll('tr')[1:]
team_stats = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]

stats_2005 = pd.DataFrame(team_stats, columns = headers)
stats_2005.head(6)

,Team,Conf,Div,W,L,W/L%,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,San Antonio Spurs,W,SW,59,23,.720,7.80,108.45,99.59,8.85,7.84,108.18,99.31,8.87
2,Phoenix Suns,W,P,62,20,.756,7.12,115.01,107.53,7.49,7.08,115.06,107.63,7.43
3,Dallas Mavericks,W,SW,58,24,.707,5.74,111.32,104.88,6.44,5.85,111.30,104.76,6.55
4,Miami Heat,E,SE,59,23,.720,6.52,111.08,103.74,7.33,5.76,110.81,104.31,6.50
5,Houston Rockets,W,SW,51,31,.622,4.04,107.03,102.57,4.46,4.26,106.85,102.15,4.70


In [55]:
stats_2005.to_csv('stats_2005.csv')

In [22]:
url = "https://www.basketball-reference.com/leagues/NBA_2004_ratings.html"
html = urlopen(url)
soup = bs(html)

soup.findAll('tr', limit=2)
headers = [th.getText() for th in soup.findAll('tr', limit=2)[1].findAll('th')]
headers = headers[1:]

rows = soup.findAll('tr')[1:]
team_stats = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]

stats_2004 = pd.DataFrame(team_stats, columns = headers)
stats_2004.head(6)

,Team,Conf,Div,W,L,W/L%,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,San Antonio Spurs,W,M,57,25,.695,7.22,103.01,94.79,8.23,7.51,102.62,94.09,8.53
2,Minnesota Timberwolves,W,M,58,24,.707,5.49,106.53,100.30,6.23,5.86,106.41,99.79,6.62
3,Sacramento Kings,W,P,55,27,.671,5.01,110.87,105.42,5.45,5.41,111.02,105.14,5.88
4,Detroit Pistons,E,C,54,28,.659,5.84,103.18,96.48,6.70,5.04,102.81,97.01,5.81
5,Indiana Pacers,E,C,61,21,.744,5.76,104.77,98.16,6.61,4.93,104.47,98.77,5.70


In [56]:
stats_2004.to_csv('stats_2004.csv')

In [23]:
url = "https://www.basketball-reference.com/leagues/NBA_2003_ratings.html"
html = urlopen(url)
soup = bs(html)

soup.findAll('tr', limit=2)
headers = [th.getText() for th in soup.findAll('tr', limit=2)[1].findAll('th')]
headers = headers[1:]

rows = soup.findAll('tr')[1:]
team_stats = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]

stats_2003 = pd.DataFrame(team_stats, columns = headers)
stats_2003.head(6)

,Team,Conf,Div,W,L,W/L%,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,Dallas Mavericks,W,M,60,22,.732,7.78,111.43,102.89,8.54,7.91,111.52,102.85,8.67
2,Sacramento Kings,W,P,59,23,.720,6.50,106.45,99.50,6.95,6.68,106.36,99.20,7.16
3,San Antonio Spurs,W,M,60,22,.732,5.41,106.25,100.29,5.96,5.65,106.20,99.99,6.22
4,New Jersey Nets,E,A,49,33,.598,5.22,104.42,98.80,5.62,4.42,103.93,99.18,4.75
5,Detroit Pistons,E,C,50,32,.610,3.68,104.84,100.71,4.14,2.98,104.48,101.11,3.36


In [57]:
stats_2003.to_csv('stats_2003.csv')

In [24]:
url = "https://www.basketball-reference.com/leagues/NBA_2002_ratings.html"
html = urlopen(url)
soup = bs(html)

soup.findAll('tr', limit=2)
headers = [th.getText() for th in soup.findAll('tr', limit=2)[1].findAll('th')]
headers = headers[1:]

rows = soup.findAll('tr')[1:]
team_stats = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]

stats_2002 = pd.DataFrame(team_stats, columns = headers)
stats_2002.head(6)

,Team,Conf,Div,W,L,W/L%,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,Sacramento Kings,W,P,61,21,.744,7.61,109.70,101.62,8.08,7.61,109.48,101.37,8.12
2,Los Angeles Lakers,W,P,58,24,.707,7.12,110.29,102.52,7.77,7.15,110.12,102.30,7.82
3,San Antonio Spurs,W,M,58,24,.707,6.21,107.35,100.38,6.97,6.27,107.06,100.01,7.05
4,Dallas Mavericks,W,M,57,25,.695,4.26,112.80,108.22,4.59,4.41,112.91,108.13,4.78
5,Minnesota Timberwolves,W,M,50,32,.610,3.38,109.79,105.97,3.82,3.58,109.78,105.73,4.05


In [58]:
stats_2002.to_csv('stats_2002.csv')

In [25]:
url = "https://www.basketball-reference.com/leagues/NBA_2001_ratings.html"
html = urlopen(url)
soup = bs(html)

soup.findAll('tr', limit=2)
headers = [th.getText() for th in soup.findAll('tr', limit=2)[1].findAll('th')]
headers = headers[1:]

rows = soup.findAll('tr')[1:]
team_stats = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]

stats_2001 = pd.DataFrame(team_stats, columns = headers)
stats_2001.head(6)

,Team,Conf,Div,W,L,W/L%,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,San Antonio Spurs,W,M,58,24,.707,7.76,107.36,98.64,8.72,7.92,107.20,98.31,8.89
2,Sacramento Kings,W,P,55,27,.671,5.82,106.32,100.23,6.08,6.07,106.23,99.85,6.38
3,Utah Jazz,W,M,53,29,.646,4.70,108.36,103.17,5.19,5.00,108.42,102.89,5.53
4,Dallas Mavericks,W,M,53,29,.646,4.28,108.00,103.36,4.63,4.60,108.07,103.07,4.99
5,Portland Trail Blazers,W,P,50,32,.610,4.20,107.08,102.54,4.53,4.52,107.11,102.21,4.90


In [59]:
stats_2001.to_csv('stats_2001.csv')

In [26]:
url = "https://www.basketball-reference.com/leagues/NBA_2000_ratings.html"
html = urlopen(url)
soup = bs(html)

soup.findAll('tr', limit=2)
headers = [th.getText() for th in soup.findAll('tr', limit=2)[1].findAll('th')]
headers = headers[1:]

rows = soup.findAll('tr')[1:]
team_stats = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]

stats_2000 = pd.DataFrame(team_stats, columns = headers)
stats_2000.head(6)

,Team,Conf,Div,W,L,W/L%,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,Los Angeles Lakers,W,P,67,15,.817,8.55,107.85,98.79,9.05,8.41,107.66,98.75,8.92
2,Portland Trail Blazers,W,P,59,23,.720,6.40,108.75,101.67,7.08,6.37,108.71,101.68,7.03
3,San Antonio Spurs,W,M,53,29,.646,5.94,105.83,99.45,6.38,5.93,105.67,99.30,6.37
4,Phoenix Suns,W,P,53,29,.646,5.22,105.04,99.66,5.38,5.24,104.90,99.48,5.41
5,Utah Jazz,W,M,55,27,.671,4.46,107.97,103.09,4.88,4.52,108.00,103.06,4.94


In [60]:
stats_2000.to_csv('stats_2000.csv')

In [27]:
url = "https://www.basketball-reference.com/leagues/NBA_1999_ratings.html"
html = urlopen(url)
soup = bs(html)

soup.findAll('tr', limit=2)
headers = [th.getText() for th in soup.findAll('tr', limit=2)[1].findAll('th')]
headers = headers[1:]

rows = soup.findAll('tr')[1:]
team_stats = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]

stats_1999 = pd.DataFrame(team_stats, columns = headers)
stats_1999.head(6)

,Team,Conf,Div,W,L,W/L%,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,San Antonio Spurs,W,M,37,13,.740,8.06,105.08,95.90,9.19,7.12,103.97,95.84,8.13
2,Utah Jazz,W,M,37,13,.740,6.52,106.47,99.06,7.41,5.54,105.27,98.96,6.31
3,Portland Trail Blazers,W,P,35,15,.700,6.36,105.54,98.50,7.03,5.67,104.49,98.24,6.26
4,Miami Heat,E,A,33,17,.660,4.96,105.76,99.91,5.85,5.11,106.23,100.23,6.00
5,Detroit Pistons,E,C,29,21,.580,3.42,105.28,101.14,4.14,3.97,106.21,101.48,4.74


In [61]:
stats_1999.to_csv('stats_1999.csv')

In [28]:
url = "https://www.basketball-reference.com/leagues/NBA_1998_ratings.html"
html = urlopen(url)
soup = bs(html)

soup.findAll('tr', limit=2)
headers = [th.getText() for th in soup.findAll('tr', limit=2)[1].findAll('th')]
headers = headers[1:]

rows = soup.findAll('tr')[1:]
team_stats = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]

stats_1998 = pd.DataFrame(team_stats, columns = headers)
stats_1998.head(6)

,Team,Conf,Div,W,L,W/L%,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,Chicago Bulls,E,C,62,20,.756,7.11,108.71,100.86,7.85,7.24,108.88,100.87,8.00
2,Los Angeles Lakers,W,P,61,21,.744,7.74,112.51,104.48,8.03,6.88,112.02,104.92,7.10
3,Indiana Pacers,E,C,58,24,.707,6.09,109.32,102.52,6.79,6.25,109.56,102.58,6.98
4,Seattle SuperSonics,W,P,61,21,.744,7.17,112.39,104.68,7.70,6.33,111.91,105.12,6.79
5,Utah Jazz,W,M,62,20,.756,6.54,113.22,105.87,7.35,5.73,112.80,106.35,6.45


In [62]:
stats_1998.to_csv('stats_1998.csv')

In [29]:
url = "https://www.basketball-reference.com/leagues/NBA_1997_ratings.html"
html = urlopen(url)
soup = bs(html)

soup.findAll('tr', limit=2)
headers = [th.getText() for th in soup.findAll('tr', limit=2)[1].findAll('th')]
headers = headers[1:]

rows = soup.findAll('tr')[1:]
team_stats = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]

stats_1997 = pd.DataFrame(team_stats, columns = headers)
stats_1997.head(6)

,Team,Conf,Div,W,L,W/L%,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,Chicago Bulls,E,C,69,13,.841,10.80,114.88,102.99,11.90,10.70,115.01,103.19,11.82
2,Utah Jazz,W,M,64,18,.780,8.79,114.41,104.75,9.66,7.97,113.94,105.23,8.72
3,Seattle SuperSonics,W,P,57,25,.695,7.68,112.02,103.67,8.35,6.91,111.50,104.03,7.47
4,Miami Heat,E,A,61,21,.744,5.49,107.67,101.38,6.30,5.56,107.71,101.31,6.40
5,Detroit Pistons,E,C,54,28,.659,5.24,111.44,105.29,6.15,5.44,111.68,105.30,6.38


In [63]:
stats_1997.to_csv('stats_1997.csv')

In [30]:
url = "https://www.basketball-reference.com/leagues/NBA_1996_ratings.html"
html = urlopen(url)
soup = bs(html)

soup.findAll('tr', limit=2)
headers = [th.getText() for th in soup.findAll('tr', limit=2)[1].findAll('th')]
headers = headers[1:]

rows = soup.findAll('tr')[1:]
team_stats = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]

stats_1996 = pd.DataFrame(team_stats, columns = headers)
stats_1996.head(6)

,Team,Conf,Div,W,L,W/L%,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,Chicago Bulls,E,C,72,10,.878,12.24,116.00,102.51,13.49,11.80,115.72,102.71,13.01
2,Seattle SuperSonics,W,P,64,18,.780,7.79,111.19,103.02,8.17,7.39,110.95,103.22,7.73
3,Utah Jazz,W,M,55,27,.671,6.59,114.00,106.64,7.37,6.24,113.95,106.98,6.96
4,San Antonio Spurs,W,M,59,23,.720,6.30,110.86,104.10,6.75,5.97,110.67,104.29,6.38
5,Orlando Magic,E,A,60,22,.732,5.56,113.57,107.76,5.81,5.39,113.58,107.93,5.65


In [64]:
stats_1996.to_csv('stats_1996.csv')

In [31]:
url = "https://www.basketball-reference.com/leagues/NBA_1995_ratings.html"
html = urlopen(url)
soup = bs(html)

soup.findAll('tr', limit=2)
headers = [th.getText() for th in soup.findAll('tr', limit=2)[1].findAll('th')]
headers = headers[1:]

rows = soup.findAll('tr')[1:]
team_stats = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]

stats_1995 = pd.DataFrame(team_stats, columns = headers)
stats_1995.head(6)

,Team,Conf,Div,W,L,W/L%,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,Utah Jazz,W,M,60,22,.732,7.99,115.18,106.48,8.70,7.75,115.00,106.58,8.43
2,Seattle SuperSonics,W,P,57,25,.695,8.18,115.51,107.06,8.45,7.91,115.34,107.19,8.16
3,Orlando Magic,E,A,57,25,.695,7.06,115.89,108.48,7.41,6.44,115.90,109.13,6.77
4,San Antonio Spurs,W,M,62,20,.756,5.96,112.31,106.04,6.27,5.90,112.13,105.93,6.19
5,Chicago Bulls,E,C,47,35,.573,4.83,110.15,105.13,5.02,4.31,110.00,105.51,4.49


In [65]:
stats_1995.to_csv('stats_1995.csv')

In [32]:
url = "https://www.basketball-reference.com/leagues/NBA_1994_ratings.html"
html = urlopen(url)
soup = bs(html)

soup.findAll('tr', limit=2)
headers = [th.getText() for th in soup.findAll('tr', limit=2)[1].findAll('th')]
headers = headers[1:]

rows = soup.findAll('tr')[1:]
team_stats = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]

stats_1994 = pd.DataFrame(team_stats, columns = headers)
stats_1994.head(6)

,Team,Conf,Div,W,L,W/L%,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,Seattle SuperSonics,W,P,63,19,.768,9.09,111.79,102.54,9.25,8.68,111.55,102.71,8.84
2,New York Knicks,E,A,57,25,.695,6.99,106.30,98.86,7.44,6.48,105.75,98.84,6.90
3,San Antonio Spurs,W,M,55,27,.671,5.26,111.19,105.49,5.70,5.05,111.31,105.83,5.47
4,Atlanta Hawks,E,C,57,25,.695,5.27,107.71,102.37,5.33,4.94,107.43,102.43,5.00
5,Phoenix Suns,W,P,56,26,.683,4.84,112.34,107.39,4.94,4.68,112.40,107.62,4.78


In [66]:
stats_1994.to_csv('stats_1994.csv')

In [33]:
url = "https://www.basketball-reference.com/leagues/NBA_1993_ratings.html"
html = urlopen(url)
soup = bs(html)

soup.findAll('tr', limit=2)
headers = [th.getText() for th in soup.findAll('tr', limit=2)[1].findAll('th')]
headers = headers[1:]

rows = soup.findAll('tr')[1:]
team_stats = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]

stats_1993 = pd.DataFrame(team_stats, columns = headers)
stats_1993.head(6)

,Team,Conf,Div,W,L,W/L%,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,Seattle SuperSonics,W,P,55,27,.671,7.07,112.89,105.72,7.17,6.66,112.71,105.98,6.73
2,Chicago Bulls,E,C,57,25,.695,6.29,113.65,107.03,6.62,6.19,113.58,107.07,6.51
3,Cleveland Cavaliers,E,C,54,28,.659,6.45,113.29,106.68,6.60,6.30,113.22,106.75,6.46
4,Phoenix Suns,W,P,62,20,.756,6.66,113.96,107.39,6.58,6.27,113.89,107.72,6.17
5,New York Knicks,E,A,60,22,.732,6.16,106.75,100.33,6.42,5.87,106.32,100.19,6.13


In [67]:
stats_1993.to_csv('stats_1993.csv')

In [34]:
url = "https://www.basketball-reference.com/leagues/NBA_1992_ratings.html"
html = urlopen(url)
soup = bs(html)

soup.findAll('tr', limit=2)
headers = [th.getText() for th in soup.findAll('tr', limit=2)[1].findAll('th')]
headers = headers[1:]

rows = soup.findAll('tr')[1:]
team_stats = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]

stats_1992 = pd.DataFrame(team_stats, columns = headers)
stats_1992.head(6)

,Team,Conf,Div,W,L,W/L%,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,Chicago Bulls,E,C,67,15,.817,10.44,116.32,105.35,10.97,10.07,116.12,105.52,10.61
2,Portland Trail Blazers,W,P,57,25,.695,7.27,111.98,104.71,7.27,6.94,111.85,104.94,6.91
3,Utah Jazz,W,M,55,27,.671,6.39,112.68,106.13,6.55,5.70,112.59,106.77,5.82
4,Phoenix Suns,W,P,53,29,.646,5.94,112.51,106.70,5.82,5.69,112.50,106.96,5.54
5,Cleveland Cavaliers,E,C,57,25,.695,5.45,114.32,108.71,5.61,5.34,114.26,108.73,5.53


In [68]:
stats_1992.to_csv('stats_1992.csv')

In [35]:
url = "https://www.basketball-reference.com/leagues/NBA_1991_ratings.html"
html = urlopen(url)
soup = bs(html)

soup.findAll('tr', limit=2)
headers = [th.getText() for th in soup.findAll('tr', limit=2)[1].findAll('th')]
headers = headers[1:]

rows = soup.findAll('tr')[1:]
team_stats = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]

stats_1991 = pd.DataFrame(team_stats, columns = headers)
stats_1991.head(6)

,Team,Conf,Div,W,L,W/L%,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,Chicago Bulls,E,C,61,21,.744,9.10,115.16,105.67,9.50,8.57,114.92,105.98,8.94
2,Portland Trail Blazers,W,P,63,19,.768,8.68,113.26,104.87,8.38,8.47,113.16,104.98,8.18
3,Los Angeles Lakers,W,P,58,24,.707,6.74,112.54,105.59,6.95,6.73,112.49,105.56,6.93
4,Phoenix Suns,W,P,55,27,.671,6.55,112.98,106.53,6.45,6.49,113.04,106.64,6.40
5,Boston Celtics,E,A,56,26,.683,5.82,112.88,107.07,5.80,5.21,112.78,107.59,5.19


In [69]:
stats_1991.to_csv('stats_1991.csv')

In [36]:
url = "https://www.basketball-reference.com/leagues/NBA_1990_ratings.html"
html = urlopen(url)
soup = bs(html)

soup.findAll('tr', limit=2)
headers = [th.getText() for th in soup.findAll('tr', limit=2)[1].findAll('th')]
headers = headers[1:]

rows = soup.findAll('tr')[1:]
team_stats = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]

stats_1990 = pd.DataFrame(team_stats, columns = headers)
stats_1990.head(6)

,Team,Conf,Div,W,L,W/L%,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,Los Angeles Lakers,W,P,63,19,.768,6.78,114.94,107.83,7.11,6.74,115.19,108.16,7.03
2,Phoenix Suns,W,P,54,28,.659,7.10,113.45,106.59,6.86,7.09,113.60,106.76,6.84
3,Portland Trail Blazers,W,P,59,23,.720,6.32,110.98,104.75,6.22,6.48,111.09,104.72,6.37
4,Detroit Pistons,E,C,59,23,.720,6.09,110.45,104.06,6.39,5.41,109.77,104.04,5.72
5,Utah Jazz,W,M,55,27,.671,4.79,110.97,106.06,4.91,4.82,111.23,106.32,4.91


In [70]:
stats_1990.to_csv('stats_1990.csv')

In [38]:
champions_df = pd.DataFrame ({
    "Year": [2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007, 2006, 2005, 2004, 2003, 2002, 2001, 2000, 1999, 1998, 1997, 1996, 1995, 1994, 1993, 1992, 1991, 1990],
    "Team": ["Toronto Raptors", "Golden State Warriors", "Golden State Warriors", "Cleveland Cavaliers", "Golden State Warriors", "San Antonio Spurs", "Miami Heat", "Miami Heat", "Dallas Mavericks", "Los Angeles Lakers", "Los Angeles Lakers", "Boston Celtics", "San Antonio Spurs", "Miami Heat", "San Antonio Spurs", "Detroit Pistons", "San Antonio Spurs", "Los Angeles Lakers", "Los Angeles Lakers", "Los Angeles Lakers", "San Antonio Spurs", "Chicago Bulls", "Chicago Bulls", "Chicago Bulls", "Houstion Rockets", "Houston Rockets", "Chicago Bulls", "Chicago Bulls", "Chicago Bulls", "Detroit Pistons"]
})
champions_df

,Year,Team
0,2019,Toronto Raptors
1,2018,Golden State Warriors
2,2017,Golden State Warriors
3,2016,Cleveland Cavaliers
4,2015,Golden State Warriors
5,2014,San Antonio Spurs
6,2013,Miami Heat
7,2012,Miami Heat
8,2011,Dallas Mavericks
9,2010,Los Angeles Lakers


In [39]:
champions_df.to_csv('champions_list.csv')